<a href="https://colab.research.google.com/github/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm/blob/oscar2/notebooks/oscar_play_osca_kernel_size_2_avgpool_replicate_resnet18_adamw_1.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone, Install Import

## Timestamp this run

In [1]:
!pip install pytz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# reference: https://www.programiz.com/python-programming/datetime/current-time

from datetime import datetime
import pytz

print(datetime.now(pytz.timezone('America/New_York')).strftime('%m/%d(%a)_%H:%M:%S'))

11/20(Sun)_10:36:05


# Wandb install, login, import

In [3]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!wandb login "6f19b1e6735ebc69af24f18d5b426262416027fb"

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
import wandb

## Torchsummary Install, Import

In [6]:
!pip install torch-summary==1.4.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from torchsummary import summary

summary

<function torchsummary.torchsummary.summary(model: torch.nn.modules.module.Module, input_data: Union[torch.Tensor, torch.Size, Sequence[torch.Tensor], Sequence[Union[int, Sequence[Any], torch.Size]], NoneType] = None, *args: Any, batch_dim: Union[int, NoneType] = 0, branching: bool = True, col_names: Union[Iterable[str], NoneType] = None, col_width: int = 25, depth: int = 3, device: Union[torch.device, NoneType] = None, dtypes: Union[List[torch.dtype], NoneType] = None, verbose: int = 1, **kwargs: Any) -> torchsummary.model_statistics.ModelStatistics>

## Clone team's code

In [8]:
!rm -r /content/csgy6953_DeepLearning_Midterm/

In [9]:
!git clone -b config_kernel_size_on_updated_main "https://github.com/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm.git"

Cloning into 'csgy6953_DeepLearning_Midterm'...
remote: Enumerating objects: 686, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 686 (delta 90), reused 76 (delta 50), pack-reused 507
Receiving objects: 100% (686/686), 319.97 KiB | 713.00 KiB/s, done.
Resolving deltas: 100% (441/441), done.


if running on Google Colab:

In [10]:
!cp -r /content/csgy6953_DeepLearning_Midterm/src/ .

if running on Amazon SageMaker Studio Lab:

In [11]:
# !cp -r csgy6953_DeepLearning_Midterm/src/ .

In [12]:
!cat src/model.py

from enum import Enum
from typing import List, Optional, Tuple

import torch
import torch.nn as nn


class ResidualBlockType(Enum):
    """
    Enum class to represent the residual block type for ResNet
    """

    BASIC = 0
    BOTTLENECK = 1


class LayerType(Enum):
    """
    Enum class to represent layer within for ResidualBlock and for BottleneckResidualBlock
    """

    # Disambiguation: here "layer" refers to the individual layer within a block,
    # not a "residual layer" containing one or more blocks
    CONV = 0


class LayerLoc(Enum):
    """
    Enum class to represent a layer's location within a block
    """

    MAIN_BLOCK_CONV1 = 0
    MAIN_BLOCK_CONV2 = 1
    MAIN_BLOCK_CONV3 = 2

    SHORTCUT_IDENTITY = 6   # identity
    SHORTCUT_CONV_STEM = 7


def generate_layer(
    block_type: ResidualBlockType,
    layer_type: LayerType, 
    layer_loc: LayerLoc, # position of this layer within the block starting from index 1 
    num_channels: int,
    main_block_kernel_siz

# Import, Seed, Device

In [13]:
import torch
import torch.nn as nn

import time
import random

In [14]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Load Data

In [16]:
from src.data import get_transformed_data, make_data_loaders
from src.transforms import make_auto_transforms # used to use: make_transforms

BATCH_SIZE = 128
VALID_RATIO = 0.1

train_data, valid_data, test_data = \
get_transformed_data(make_transforms=make_auto_transforms, valid_ratio=VALID_RATIO)

train_iter, valid_iter, test_iter = \
make_data_loaders(train_data, valid_data, test_data, BATCH_SIZE)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Model

## Model: import

In [17]:
from src.model import StemConfig, ResidualBlockType, ResNet

## Model: try code here
`config_kernel_size_on_updated_main` branch > src > model.py

## Architecture <<<

In [18]:
DROPOUT = 0.1
MAIN_BLOCK_KERNEL_SIZE = 2

# 1 tuple == 1 layer
# block with or without bottleneck, how many blocks in each layer, out_channels, dropout prob, kernel_size
architecture = [
    (ResidualBlockType.BASIC, 2,  64, DROPOUT, MAIN_BLOCK_KERNEL_SIZE),
    (ResidualBlockType.BASIC, 2, 128, DROPOUT, MAIN_BLOCK_KERNEL_SIZE),
    (ResidualBlockType.BASIC, 2, 256, DROPOUT, MAIN_BLOCK_KERNEL_SIZE),
    (ResidualBlockType.BASIC, 2, 512, DROPOUT, MAIN_BLOCK_KERNEL_SIZE),
]

# 2022.11/18(5)_p03.59 (Oscar)
# slightly reduce the last layer's out_channels to keep overall params under 5M:
# 512 -> 5,069,130
# 508 -> 5,014,978
# 507 -> 5,001,500
# 506 -> 4,988,046 <<<

In [19]:
config = StemConfig(num_channels=64, kernel_size=3, stride=1, padding=1)

model  = ResNet(architecture, stem_config=config, output_size=10)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


## Initialize model shapes

In [20]:
# intialize a new model

inputs = torch.empty((BATCH_SIZE, 3, 32, 32)) #  passed
inputs.normal_()

model = model.to(device)

outputs = model(inputs.to(device)) # internally converts all nn.LazyConv2d layers to nn.Conv2d

print('-------------------')
print('-------------------')

print(model)

from src.utils import count_parameters

print(count_parameters(model))
print(outputs.size())

-------------------
-------------------
ResNet(
  (stem): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (classifier): Sequential(
    (0): AdaptiveAvgPool2d(output_size=1)
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
  (body): Sequential(
    (block_2): Sequential(
      (0): ResidualBlock(
        (dropout): Dropout(p=0.1, inplace=False)
        (conv1): Sequential(
          (0): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), bias=False)
          (1): AvgPool2d(kernel_size=2, stride=1, padding=0)
        )
        (conv2): Sequential(
          (0): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), bias=False)
          (1): AvgPool2d(kernel_size=2, stride=1, padding=0)
        )
        (relu): ReLU(inplace=True)
 

## Count parameters

In [21]:
num_parameters, num_parameters_requiring_grad = count_parameters(model)
print(num_parameters)

5069130


In [22]:
QUIET_VERBOSE = 0

one_sample_input_shape = (3, 32, 32)

summary(model, one_sample_input_shape, verbose = QUIET_VERBOSE)

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 32, 32]          --
|    └─Conv2d: 2-1                       [-1, 64, 32, 32]          1,728
|    └─BatchNorm2d: 2-2                  [-1, 64, 32, 32]          128
|    └─ReLU: 2-3                         [-1, 64, 32, 32]          --
├─Sequential: 1-2                        [-1, 512, 4, 4]           --
|    └─Sequential: 2-4                   [-1, 64, 32, 32]          --
|    |    └─ResidualBlock: 3-1           [-1, 64, 32, 32]          33,024
|    |    └─ResidualBlock: 3-2           [-1, 64, 32, 32]          33,024
|    └─Sequential: 2-5                   [-1, 128, 16, 16]         --
|    |    └─ResidualBlock: 3-3           [-1, 128, 16, 16]         107,008
|    |    └─ResidualBlock: 3-4           [-1, 128, 16, 16]         131,584
|    └─Sequential: 2-6                   [-1, 256, 8, 8]           --
|    |    └─ResidualBlock: 3-5           [-1, 256, 8, 8]       

## Initialize parameters

In [23]:
from src.utils import initialize_parameters

# initialize model weights
model.apply(initialize_parameters);

# Train, validate, log

## Loss function

In [24]:
criterion = nn.CrossEntropyLoss().to(device)

## Optimizer, scheduler (learning rate) <<<

In [25]:
init_learning_rate = 0.01

optimizer = torch.optim.AdamW(model.parameters(), lr=init_learning_rate)



from torch.optim.lr_scheduler import CyclicLR

# base_lr, max_lr, lr_schedule = 0.2, 1.1, 'CyclicLR_base_0.2_max_1.1'
base_lr, max_lr, lr_schedule = 0.02, 1.1, 'CyclicLR_base_0.02_max_1.1'

scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr, cycle_momentum=False)

## Iterate!

In [26]:
from src.engine import train_one_epoch, evaluate

In [27]:
EPOCHS  = 100

In [28]:
## setup wandb logging

assert 'wandb' in locals() # 'wandb' has been imported already

ENTITY_NAME  = 'dlf22_mini_project' # this is our team name

# PROJECT_NAME = 'Junk_Test_Project'
PROJECT_NAME = 'ResNet_5M'

# RUN_NAME     = 'resnet_osca_kernel_size_2_2022.11.18.p04.13'
# RUN_NAME = 'osca_kernel_size_2_avgpool_replicate_resnet18_adamw_0.2baselr'
RUN_NAME = 'osca_kernel_size_2_avgpool_replicate_resnet18_adamw_0.02baselr'

WANDB_CONFIG = \
{"architecture"          : architecture,                  # the model
 "num_params"            : num_parameters,                # the model
 "grad_params"           : num_parameters_requiring_grad, # the model
 "main_block_kernel_size": MAIN_BLOCK_KERNEL_SIZE,        # the model
 "dropout"               : DROPOUT,                       # the model
 "init_learning_rate" : init_learning_rate, # how to gradient descent
 "base_lr"            : base_lr,            # how to gradient descent
 "lr_schedule"        : lr_schedule,        # how to gradient descent
 "max_lr"             : max_lr,             # how to gradient descent
 "batch_size"         : BATCH_SIZE,         # how to gradient descent
 "epochs"             : EPOCHS,}   # train for how long

WANDB_CONFIG

{'architecture': [(<ResidualBlockType.BASIC: 0>, 2, 64, 0.1, 2),
  (<ResidualBlockType.BASIC: 0>, 2, 128, 0.1, 2),
  (<ResidualBlockType.BASIC: 0>, 2, 256, 0.1, 2),
  (<ResidualBlockType.BASIC: 0>, 2, 512, 0.1, 2)],
 'num_params': 5069130,
 'grad_params': 5069130,
 'main_block_kernel_size': 2,
 'dropout': 0.1,
 'init_learning_rate': 0.01,
 'base_lr': 0.02,
 'lr_schedule': 'CyclicLR_base_0.02_max_1.1',
 'max_lr': 1.1,
 'batch_size': 128,
 'epochs': 100}

In [29]:
wandb\
.init(name=RUN_NAME, project=PROJECT_NAME, entity=ENTITY_NAME,config=WANDB_CONFIG)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: os2000os (dlf22_mini_project). Use `wandb login --relogin` to force relogin


In [30]:
# save to disk the "best" model === the model with the lowest validation loss
best_model_path = RUN_NAME + '.pt'
print(best_model_path)

osca_kernel_size_2_avgpool_replicate_resnet18_adamw_0.02baselr.pt


In [31]:
from src.utils import epoch_time

FIRST_EPOCH = 1

best_loss = float('inf')
for epoch in range(FIRST_EPOCH, FIRST_EPOCH + EPOCHS):

    print(f"Epoch {epoch}")
    
    ## TRAIN
    start = time.time()
    # train
    train_loss, train_acc  = train_one_epoch(model, train_iter, criterion, optimizer, device)
    # log & echo
    train_mins, train_secs = epoch_time(start, time.time())
    wandb.log({"train_loss": train_loss,
               "train_acc" : train_acc,
               "epoch"     : epoch})
    print(f"\tTrain elapsed: {train_mins}:{train_secs}, loss: {train_loss:.4f}, acc: {train_acc * 100:.2f}%")

    ## VALIDATE
    start = time.time()
    # validate
    val_loss, val_acc  = evaluate(model, valid_iter, criterion, device)
    # log & echo   
    val_mins, val_secs = epoch_time(start, time.time())
    wandb.log({"val_loss": val_loss,
               "val_acc" : val_acc,
               "epoch"   : epoch,})
    print(f"\tValidation elapsed: {val_mins}:{val_secs}, loss: {val_loss:.4f}, acc: {val_acc * 100:.2f}%")
    
    scheduler.step()
    
    current_lr = scheduler.get_last_lr()[0]
    wandb.log({"current_lr": current_lr,
               "current_learning_rate": current_lr,
               "epoch"     : epoch})
    
    ## Memorize "best" model === the model with the lowest validation loss
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), best_model_path)

Epoch 1
	Train elapsed: 1:6, loss: 2.7274, acc: 16.64%
	Validation elapsed: 0:1, loss: 1.9141, acc: 25.57%
Epoch 2
	Train elapsed: 1:6, loss: 1.8774, acc: 30.06%
	Validation elapsed: 0:1, loss: 1.6366, acc: 38.67%
Epoch 3
	Train elapsed: 1:6, loss: 1.7312, acc: 36.23%
	Validation elapsed: 0:1, loss: 1.5558, acc: 41.91%
Epoch 4
	Train elapsed: 1:6, loss: 1.5957, acc: 41.97%
	Validation elapsed: 0:1, loss: 1.3256, acc: 50.64%
Epoch 5
	Train elapsed: 1:6, loss: 1.5019, acc: 45.49%
	Validation elapsed: 0:1, loss: 1.2963, acc: 52.85%
Epoch 6
	Train elapsed: 1:6, loss: 1.4177, acc: 48.90%
	Validation elapsed: 0:1, loss: 1.3282, acc: 50.90%
Epoch 7
	Train elapsed: 1:6, loss: 1.3467, acc: 51.73%
	Validation elapsed: 0:1, loss: 1.1296, acc: 59.88%
Epoch 8
	Train elapsed: 1:6, loss: 1.2957, acc: 53.67%
	Validation elapsed: 0:1, loss: 1.1337, acc: 59.55%
Epoch 9
	Train elapsed: 1:6, loss: 1.2527, acc: 55.28%
	Validation elapsed: 0:1, loss: 1.3688, acc: 50.94%
Epoch 10
	Train elapsed: 1:6, loss: 1

KeyboardInterrupt: ignored

# Test, log

In [32]:
print(best_model_path),
model.load_state_dict(torch.load(best_model_path))

osca_kernel_size_2_avgpool_replicate_resnet18_adamw_0.02baselr.pt


<All keys matched successfully>

In [33]:
test_loss, test_acc = evaluate(model.to(device), test_iter, criterion, device)
print(f"Test Loss: {test_loss:.4f}\nTest Accuracy: {test_acc * 100:.2f}%")

wandb.log({
    "test_loss": test_loss,
    "test_acc" : test_acc,
})

Test Loss: 0.8516
Test Accuracy: 69.82%


In [34]:
print(datetime.now(pytz.timezone('America/New_York')).strftime('%m/%d(%a)_%H:%M:%S'))

11/20(Sun)_11:14:12
